In [77]:
import time
import requests
import datetime
import lxml
from urllib.parse import urljoin

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import codecs
import re
import json

import sqlite3
import chromadb

In [78]:
def get_news_date(date_format="%Y%m%d"):
    """
    Determines the "news date" based on the current time.

    The "news day" is considered to change at 10 PM (22:00).
    - If the current time is 10 PM or later, it returns today's date.
    - If the current time is before 10 PM, it returns yesterday's date.

    Args:
        date_format (str, optional): The format for the output date string,
                                     using standard strftime codes.
                                     Defaults to "%Y%m%d".

    Returns:
        str: The formatted date string for the relevant news day.
    """
    # Get the current date and time
    now = datetime.datetime.now()
    
    # The cutoff hour is 10 PM, which is 22 in 24-hour format
    cutoff_hour = 22
    
    # Check if the current hour is past the cutoff time
    if now.hour >= cutoff_hour:
        # It's 10 PM or later, so the news day is today
        target_date = datetime.date.today()
    else:
        # It's before 10 PM, so we should be looking at yesterday's news
        target_date = datetime.date.today() - datetime.timedelta(days=1)
        
    # Format the determined date into the desired string format
    return target_date.strftime(date_format)

In [79]:
# Prep hard codes and session
date = get_news_date()
kbs_program_url = f"https://news.kbs.co.kr/news/pc/program/program.do?bcd=0001&ref=pGnb#{date}"
mbc_program_url = f"https://imnews.imbc.com/replay/2025/nwdesk/"
sbs_program_url = f"https://news.sbs.co.kr/news/programMain.do?prog_cd=R1&broad_date={date}&plink=CAL&cooper=SBSNEWS"

session = requests.Session()

In [80]:
# Selenium setup
options = webdriver.FirefoxOptions()
options.add_argument("--headless")

geckodriver_path = "/home/user0/tmp/geckodriver"
service = webdriver.FirefoxService(executable_path=geckodriver_path)
driver = webdriver.Firefox(service=service, options=options)

In [14]:
def get_kbsnews(url, session):
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    script_tag = soup.find('script', string=re.compile(r"var messageText"))
    pattern = re.compile(r'var messageText = "(.*?)";', re.DOTALL)
    match = pattern.search(script_tag.text)
    content = match.group(1)
    news_soup = BeautifulSoup(content, 'lxml')
    text = news_soup.get_text(separator="\n", strip=True)
    text = text.replace("\\", "")
    pattern = r"\nKBS 뉴스 [가-힣]+입니다\.[\s\S]*"
    cleaned_text = re.sub(pattern, "", text).strip()
    return cleaned_text

In [27]:
def get_kbs_newslist(date, kbs_program_url, driver, session):
    # Get Selenium driver response of kbs news url
    driver.get(kbs_program_url)
    wait = WebDriverWait(driver, 15)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "box-content")))

    # Get html elements containing label box-content
    kbs_source = driver.page_source
    kbs_soup = BeautifulSoup(kbs_source, 'lxml')
    kbs_items = kbs_soup.select("a.box-content")

    # make initial list
    kbs_base_url = "https://news.kbs.co.kr"
    kbs_boxlist = []
    for item in kbs_news_items:
        title = item.find('p', class_='title').get_text(strip=True) if item.find('p', class_='title') else "N/A"
        relative_link = item.get('href')
        full_link = urljoin(kbs_base_url, relative_link)
        kbs_boxlist.append({'title': title, 'url': full_link, 'broadcast_date': date})

    # Remove empty elements and sports
    isNews = False
    kbs_newslist = []
    for i in range(len(kbs_boxlist)):
        if kbs_boxlist[i]['title'] == '오프닝':
            isNews = True
            continue
        if isNews == False:
            continue
        else:
            if kbs_boxlist[i]['title'] == '[스포츠9 헤드라인]':
                isNews = False
                continue
            kbs_newslist.append(kbs_boxlist[i])

    # add index 
    for i in range(len(kbs_newslist)):
        kbs_newslist[i]['order'] = i + 1

    # add news
    for i in range(len(kbs_newslist)):
        news = get_kbsnews(kbs_newslist[i]['url'], session)
        kbs_newslist[i]['news'] = news
        
kbs_newslist


[{'title': '민주당 “조희대 사퇴” 총공세…“사법부 신뢰 잃어”',
  'url': 'https://news.kbs.co.kr/news/pc/view/view.do?ncd=8358150',
  'broadcast_date': '20250916',
  'order': 1,
  'news': '[앵커]\n여권의 사법부 압박이 더욱 거세졌습니다.\n오늘(15일) 더불어민주당 지도부가 조희대 대법원장에게 사퇴하라고 공개적으로 요구했습니다.\n대법원장이 사퇴하지 않으면 탄핵해야 한다는 말까지 나왔습니다.\n첫 소식, 이예린 기자의 보도입니다.\n[리포트]\n"정치적 중립성을 의심 받고 있는 대법원장은 사퇴해야 한다" 민주당 지도부가 조희대 대법원장의 사퇴 요구를  공식화했습니다.\n대법원이 지난 5월 이재명 당시 대선 후보 사건을  파기환송 하면서  사법부의 신뢰를 잃게 했는데 조 대법원장이 책임지란 겁니다.\n[정청래/더불어민주당 대표 : "재판 독립, 법원의 정치적 중립은 조희대 대법원장 본인 스스로 어긴 것 아닙니까? 지금이라도 사퇴하는 게 맞다고…."]\n[김병주/더불어민주당 최고위원 : "대통령 후보 바꾸기를 획책하더니 내란 심판에는 \'재판독립\' 운운하는 조희대 대법원장부터 사퇴해야…."]\n조 대법원장이 내란범을 재판 지연으로 보호하고 있다, 사법 독립을 위해서 물러나야 한다 등 사퇴 요구가 분출했습니다.\n사퇴하지 않는다면 탄핵해야 한다는 주장도 나왔습니다.\n[서영교/더불어민주당 의원/YTN라디오 \'더 인터뷰\' : "대선에 개입하지 않았습니까? 법률과 헌법을 위반했다. 이렇게 판단하고 있고, 탄핵의 대상이다."]\n내란 전담재판부 설치 필요성도 거듭 제기했는데, 조 대법원장의 정치적 편향성이 불러온 자업자득이라고 지적했습니다.\n나아가 김건희 여사 의혹에 대한 \'국정농단 전담재판부\'를 도입해야 한다는 주장도 나왔습니다.\n[장경태/더불어민주당 의원 : "내란과 국정농단에 대한 빠른 종식에 대해서는 여러 재판부에 대한 교체, 재판부에 대

In [34]:
def get_mbcnews(url, session):
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    article_divs = soup.select_one("div.news_txt")
    text = article_divs.get_text(separator="\n", strip=True)
    pattern = r"\nMBC뉴스 [가-힣]+입니다\.[\s\S]*"
    cleaned_text = re.sub(pattern, "", text).strip()
    pattern1 = r"\nMBC 뉴스 [가-힣]+입니다\.[\s\S]*"
    cleaned_text = re.sub(pattern1, "", cleaned_text).strip()
    return cleaned_text

In [110]:
def get_mbcnewslist(date, mbc_program_url, driver, session):
    # Get Selenium driver response of mbc news url
    driver.get(mbc_program_url)
    wait = WebDriverWait(driver, 15)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "item")))

    # Get html elements containing label box-content
    mbc_source = driver.page_source
    soup = BeautifulSoup(mbc_source, 'lxml')
    mbc_news_html = soup.select("li.item")

    # get newslist, which contains title, and url of news. note that it is yet unsanitized.
    session = requests.Session()
    mbc_newslist = []
    for item in mbc_news_html:
        title = None
        if item.find('span', class_='tit ellipsis2'):
            title = item.find('span', class_='tit ellipsis2').get_text(strip=True)
        elif item.find('span', class_='tit ellipsis'):
            title = item.find('span', class_='tit ellipsis').get_text(strip=True)
        else:
            "N/A"
        if (title.startswith('[톱플레이]')):
            break
        link = item.find('a').get('href')
        news = get_mbcnews(link, session)
        mbc_newslist.append({'title': title, 'url': link, 'broadcast_date': date, 'news': news})
    
    for i in range(len(mbc_newslist)):
        mbc_newslist[i]['order'] = i + 1

    return mbc_newslist

mbc_newslist

[{'title': '9개월 만에 내놓은 대책이 판사 1명 추가? 시기·실효성 모두 의문',
  'url': 'https://imnews.imbc.com/replay/2025/nwdesk/article/6757454_36799.html',
  'broadcast_date': '20250916',
  'news': '◀ 앵커 ▶\n납득하기 힘든 결정과 더딘 진행으로 내란 사건 재판부에 대한 비판이 이어지는 가운데 법원이 오늘에서야 자구책을 내놨습니다.\n내란 사건 재판부에 판사 1명을 추가하겠다는 건데요.\n\'친위 쿠데타\'가 발생한 지 무려 9개월, 윤 전 대통령이 기소된 지는 약 8개월 만에 나온 조치입니다.\n유서영 기자가 보도합니다.\n◀ 리포트 ▶\n내란 사건 재판이 진행되고 있는 서울중앙지법에서 예고 없이 갑자기 3대 특검 사건 재판에 대한 지원책을 내놨습니다.\n먼저 내란 사건을 담당하고 있는 형사합의25부에 판사 한 명을 추가 배치하기로 했습니다.\n새로 투입되는 판사가 일반 사건을 맡도록 해 지귀연 재판장을 포함한 기존의 판사 3명이 내란 재판에 좀 더 집중할 수 있도록 하겠다는 겁니다.\n또 특검 사건 한 건이 배당되면 일반 사건 다섯 건을 배당하지 않는 방식으로 3대 특검 사건을 맡게 되는 재판부의 부담을 줄여주기로 했습니다.\n지난해 말, 내란사태를 주도한 혐의로 재판에 넘겨진 김용현 전 국방장관의 1심 재판부가 지귀연 재판장이 있는 형사합의 25부로 정해진 지 9개월만입니다.\n하지만 왜 이제서야 이런 지원책을 내놨는지는 의문입니다.\n서울중앙지법은 앞서 조지호 경찰청장, 윤석열 전 대통령 등이 기소될 때 효율적인 재판 진행을 위해 사건을 해당 재판부로 모으고 신규 사건 배당도 중단했습니다.\n\'친위 쿠데타\'라는 대형 사건으로 곧 재판이 물밀듯이 들어올 상황을 예상하고 있었다는 겁니다.\n그러나 1심 재판이 끝나기도 전에 윤 전 대통령의 구속기간이 끝나 풀려날 수 있다는 비판까지 제기됐습니다.\n수도권의 한 현직 판사는 "\'양승태 

In [115]:
def get_sbsnews(url, session):
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    script_tag = soup.find('script', type='application/ld+json')
    if script_tag:
        # Get the string content of the script tag
        json_text = script_tag.string
        
        # 4. Parse the JSON text into a Python dictionary
        data = json.loads(json_text)
        
        # 5. Access the "articleBody" value
        article_body = data.get("articleBody")
        
    return article_body

In [116]:
def get_sbsnewslist(date, sbs_program_url, driver, session):
    # Get Selenium response of sbs url
    driver.get(sbs_program_url)
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li[itemprop="itemListElement"]')))
    
    sbs_source = driver.page_source
    soup = BeautifulSoup(sbs_source, 'lxml')
    sbs_news_html = soup.select('li[itemprop="itemListElement"]')
    
    sbs_base_url = "https://news.sbs.co.kr"
    sbs_newslist = []
    for item in sbs_news_html:
        category_tag = item.find("em", class_="cate")
        if category_tag and category_tag.get_text(strip=True) == "스포츠":
            continue
        title = item.find('img').get('alt')
        relative_link = item.find('a').get('href')
        full_link = urljoin(sbs_base_url, relative_link)
        news = get_sbsnews(full_link, session)
    
        if title.startswith('[날씨]'):
            break
    
        sbs_newslist.append({'title': title, 'url': full_link, 'broadcast_date': date, 'news': news})
        
    for i in range(len(sbs_newslist)):
        sbs_newslist[i]['order'] = i + 1   
        

    return sbs_newslist

In [118]:
sbs_newslist = get_sbsnewslist(date, sbs_program_url, driver, session)
sbs_newslist

[{'title': 'KT 소액결제 피해, 서초·동작·일산에서도 확인',
  'url': 'https://news.sbs.co.kr/news/endPage.do?news_id=N1008265925&plink=THUMB&cooper=SBSNEWSPROGRAM',
  'broadcast_date': '20250921',
  'news': '<앵커> 일요일 8시 뉴스는 KT 해킹 사고 관련 소식으로 시작합니다. KT 이용자들의 휴대전화 무단 소액결제 피해 규모가 불어나고 있습니다. 기존에 알려진 서울 서남권과 경기 일부 지역 말고도 서울 서초와 동작구, 고양 일산동구에서도 추가 피해가 발생한 걸로 확인됐습니다. 오늘(21일) 첫 소식, 동은영 기자입니다. <기자> KT의 소액결제 피해는 이동 가능한 불법 기지국 \'펨토셀\'을 이용해 접속 이용자들의 휴대전화를 해킹한 뒤, 모바일 상품권을 구매하거나 교통카드를 충전하는 방식으로 일어났습니다. 그런데, KT가 국회에 제출한 자료에 따르면 기존에 알려진 경기 광명과 부천, 서울 금천구 등 수도권 서남부 이외에도 피해 지역이 더 있었던 것으로 확인됐습니다. 지난달 5일부터 8일 사이 서울 동작구, 관악구, 영등포구에서 15명이 900여만 원의 피해를 입었고, 8일과 11일에는 서울 서초구에서 3명이 227만 원의 소액결제 피해를 봤습니다. 이후에는 경기 광명시와 서울 금천구에서 26명의 피해가 잇따랐습니다. 그리고 같은 달 20일 고양시 일산동구에서 3명이 170여만 원의 피해를 봤습니다. 피해 장소는 \'소액결제 전 해킹으로 의심되는 인증이 이뤄진 곳\'을 뜻하는데, 서울 강남과 경기 북부까지 수도권 전역으로 확대된 겁니다. KT는 또 피해 고객 수를 앞서 밝힌 278명보다 많은 362명으로 정정했습니다. KT는 "펨토셀과 신호를 주고받은 고객 2만여 명에 대해 조사를 마쳤다"며 "피해는 ARS 인증을 통해서만 이뤄진 것으로 파악됐다"고 밝혔습니다. [김승주/고려대 정보보호대학원 교수 : 2만여 명이라는 수치는 불법 초